<a href="https://colab.research.google.com/github/n-a-peter/MLZ-homework/blob/main/08_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
!unzip data.zip

--2025-12-01 21:48:02--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-01T22%3A38%3A04Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-01T21%3A37%3A50Z&ske=2025-12-01T22%3A38%3A04Z&sks=b&skv=2018-11-09&sig=s4cZmc%2BEw55ExXiXl6jvgEnDOEc28zxRXVrRxWNSpa4%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDYyNzQxNCwibmJmIjoxNzY0NjI1NjE0LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5i

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms

import os
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [87]:

class HairDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(data_dir))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        for label_name in self.classes:
            label_dir = os.path.join(data_dir, label_name)
            for img_name in os.listdir(label_dir):
                self.image_paths.append(os.path.join(label_dir, img_name))
                self.labels.append(self.class_to_idx[label_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [88]:
batch_size = 20
num_classes = 1
size_inner=64
learning_rate = 0.002
num_epochs = 10

# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [162]:
input_size = 200

# ImageNet normalization values
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Simple transforms - just resize and normalize
train_transforms = transforms.Compose([
    #transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
])

test_transforms = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [90]:

train_dataset = HairDataset(
    data_dir='./data/train',
    transform=train_transforms
)

test_dataset = HairDataset(
    data_dir='./data/test',
    transform=test_transforms
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [139]:
# Build the model

# Creating a CNN class
class HairClassifierNeuralNet(nn.Module):
#  Determine what layers and their order in CNN object
    def __init__(self, size_inner=64, num_classes=1):
        super(HairClassifierNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=0, stride=1)
        self.relu = nn.ReLU()
        self.max_pool1 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(313632, size_inner)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(size_inner, num_classes)
        #self.sigmoid = nn.Sigmoid()

    # Progresses data across layers
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.relu(out)
        out = self.max_pool1(out)
        out = torch.flatten(out, 1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        #out = self.sigmoid(out)
        return out

In [145]:
def make_model(learning_rate=learning_rate, size_inner=size_inner):
    model = HairClassifierNeuralNet(
        num_classes=num_classes,
        size_inner=size_inner
    )
    model.to(device)
    criterion = nn.BCEWithLogitsLoss() #nn.CrossEntropyLoss() # nn.BCEWithLogitsLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.8)
    return model, optimizer, criterion

In [146]:
model, optimizer, criterion = make_model(learning_rate=learning_rate, size_inner=size_inner)

In [148]:
# Option 1: Using torchsummary (install with: pip install torchsummary)
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

# Option 2: Manual counting
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
              ReLU-2         [-1, 32, 198, 198]               0
         MaxPool2d-3           [-1, 32, 99, 99]               0
            Linear-4                   [-1, 64]      20,072,512
              ReLU-5                   [-1, 64]               0
            Linear-6                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 21.54
Params size (MB): 76.57
Estimated Total Size (MB): 98.57
----------------------------------------------------------------
Total parameters: 20073473


In [149]:
len(train_loader), len(train_dataset), len(test_loader), len(test_dataset)

(41, 801, 11, 201)

In [169]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.0013, Acc: 1.0000, Val Loss: 1.2006, Val Acc: 0.7264
Epoch 2/10, Loss: 0.0013, Acc: 1.0000, Val Loss: 1.2074, Val Acc: 0.7264
Epoch 3/10, Loss: 0.0013, Acc: 1.0000, Val Loss: 1.2009, Val Acc: 0.7214
Epoch 4/10, Loss: 0.0012, Acc: 1.0000, Val Loss: 1.2034, Val Acc: 0.7264
Epoch 5/10, Loss: 0.0011, Acc: 1.0000, Val Loss: 1.2154, Val Acc: 0.7264
Epoch 6/10, Loss: 0.0011, Acc: 1.0000, Val Loss: 1.2179, Val Acc: 0.7264
Epoch 7/10, Loss: 0.0010, Acc: 1.0000, Val Loss: 1.2159, Val Acc: 0.7264
Epoch 8/10, Loss: 0.0010, Acc: 1.0000, Val Loss: 1.2284, Val Acc: 0.7264
Epoch 9/10, Loss: 0.0009, Acc: 1.0000, Val Loss: 1.2414, Val Acc: 0.7264
Epoch 10/10, Loss: 0.0009, Acc: 1.0000, Val Loss: 1.2354, Val Acc: 0.7264


In [170]:
training_accuracy = np.sort(history['acc'])
np.median(training_accuracy)

np.float64(1.0)

In [171]:
training_loss = np.sort(history['loss'])
np.std(training_loss)

np.float64(0.00014621835086334854)

In [172]:
mean_testLoss = np.mean(history['val_loss'])
mean_testLoss

np.float64(1.216671327197227)

In [174]:
avearage_testAccuracy6to10 = np.mean(history['val_acc'][5:])
avearage_testAccuracy6to10

np.float64(0.7263681592039801)